In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Read the data
data = pd.read_csv("dataset.csv")

# Convert the 'ActivityDate' column to datetime format
data['ActivityDate'] = pd.to_datetime(data['ActivityDate'], format='%m/%d/%Y')

# Convert the 'ActivityDate' column to numeric format
data['ActivityDate'] = pd.to_numeric(data['ActivityDate'])
# Dropped because it contains many zero values
data.drop('LoggedActivitiesDistance', axis=1, inplace=True)

# Select only columns with numeric data types
numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns

# Fill missing values with the mean of numeric columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

In [30]:
# Separate features and target variable
X = data.drop("GoalAchieved", axis=1)
y = data["GoalAchieved"]

In [31]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [32]:
# Define the models
models = {
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "KNN": KNeighborsClassifier()
}

In [33]:
# Perform k-fold cross-validation
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
results = {model_name: [] for model_name in models}

for train_index, val_index in skf.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        results[model_name].append(accuracy)

C:\Users\murph\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\murph\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

In [34]:
# Compute average performance metrics
average_accuracies = {model_name: np.mean(accuracies) for model_name, accuracies in results.items()}
best_model_name = max(average_accuracies, key=average_accuracies.get)
best_model = models[best_model_name]

# Retrain the best model on the entire dataset
best_model.fit(X, y)

DecisionTreeClassifier()

In [35]:
# Print the results for each model
for model_name, accuracies in results.items():
    print(f"{model_name}:")
    print(f"  Accuracies: {accuracies}")
    print(f"  Average accuracy: {np.mean(accuracies):.4f}\n")


Logistic Regression:
  Accuracies: [0.9787234042553191, 1.0, 0.9893617021276596, 1.0, 0.9893617021276596, 0.9468085106382979, 0.9787234042553191, 0.9787234042553191, 0.9787234042553191, 0.9893617021276596]
  Average accuracy: 0.9830

SVM:
  Accuracies: [0.9361702127659575, 0.9042553191489362, 0.9468085106382979, 0.9574468085106383, 0.9148936170212766, 0.9574468085106383, 0.9787234042553191, 0.9893617021276596, 0.9148936170212766, 0.9574468085106383]
  Average accuracy: 0.9457

Decision Tree:
  Accuracies: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
  Average accuracy: 1.0000

KNN:
  Accuracies: [0.851063829787234, 0.9042553191489362, 0.9148936170212766, 0.925531914893617, 0.8723404255319149, 0.925531914893617, 0.925531914893617, 0.9574468085106383, 0.8617021276595744, 0.9468085106382979]
  Average accuracy: 0.9085

